In [ ]:
PLACEHOLDER_DATASET_PATH = "../datasets/mnist.npz"
PLACEHOLDER_MODEL_SAVE_PATH = "activation.h5"

In [ ]:
## Init
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dense, Activation, InputLayer

DATASET_PATH = PLACEHOLDER_DATASET_PATH
with np.load(DATASET_PATH, allow_pickle=True) as f:
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
x_train = x_train / 255
x_test = x_test / 255
input_shape = x_train.shape[1:]
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
## ModelConfig
model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=input_shape))
model.add(Activation("relu"))
model.add(Conv2D(32, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dense(200))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(
    loss = "categorical_crossentropy",
    optimizer = "rmsprop",
    metrics = ["accuracy"]
)

In [ ]:
## Run
history = model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test), verbose=1)
print(history.history["val_acc"][-1])

from keras.models import Model
render_layers = [model.output]
prev_layer = ""
for layer in model.layers:
    if "activation" in layer.name and ("conv2d" in prev_layer or "max_pooling" in prev_layer):
        render_layers.append(layer.output)
    prev_layer = layer.name
render_model = Model(inputs=model.input, outputs=render_layers)
render_model.save(PLACEHOLDER_MODEL_SAVE_PATH)